In [10]:
#!ngrok config add-authtoken 2gGtpjM3ivyQ8lfhWJOsyPDvqj2_6KpKBT71RvvFbv8zQVcgD

In [11]:
#!ngrok http http://localhost:8501

In [6]:
!nvidia-smi

Sat Jun 15 09:49:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0 Off |                  Off |
|  0%   51C    P2             73W /  450W |   24073MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM

app = Flask(__name__)

# ล้างหน่วยความจำ GPU ก่อนโหลดโมเดล
torch.cuda.empty_cache()

# แจ้งการโหลดโมเดลและ tokenizer
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-72B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-72B")
print("Model and tokenizer loaded.")

# ย้ายโมเดลไปยัง GPU ถ้ามี
if torch.cuda.is_available():
    model = model.to("cuda")
    print("Model moved to GPU.")
else:
    print("GPU not available, using CPU.")

@app.route('/')
def index():
    return 'Hello, World!'

@app.route('/generate', methods=['POST'])
def generate():
    torch.cuda.empty_cache()  # ล้างหน่วยความจำก่อนการประมวลผลใหม่
    text = request.form['input_text']
    inputs = tokenizer.encode(text, return_tensors="pt")
    
    # ย้าย tensors ไปยัง GPU ถ้ามี
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")

    outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    torch.cuda.empty_cache()  # ล้างหน่วยความจำหลังจากการประมวลผล
    return jsonify({'generated_text': generated_text})

if __name__ == '__main__':
    app.run(port=8501)


Loading model and tokenizer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00001-of-00037.safetensors:   0%|          | 10.5M/3.76G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/0d/ee/0deecfb2b45a159252701d3bf1cb56f185d30cc6e44e2f5899d977f0c84fe140/60ea416808b446ee22e0912107f316e17075848bb5baa4630cf78244ae6f1b07?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00037.safetensors%3B+filename%3D%22model-00001-of-00037.safetensors%22%3B&Expires=1718680823&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODY4MDgyM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBkL2VlLzBkZWVjZmIyYjQ1YTE1OTI1MjcwMWQzYmYxY2I1NmYxODVkMzBjYzZlNDRlMmY1ODk5ZDk3N2YwYzg0ZmUxNDAvNjBlYTQxNjgwOGI0NDZlZTIyZTA5MTIxMDdmMzE2ZTE3MDc1ODQ4YmI1YmFhNDYzMGNmNzgyNDRhZTZmMWIwNz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H58SrjQGUkdy9BspopdcSPYtthtuMp6xvp6YQfOv-kqv-DMMNI2zOBH6vRR5ZqimOHL3PgTZo-7FvABDmFyYGxNGUoUm7P2Qu4T%7Ek-UJIH2YkrUjQgdD1M-vP0uU7%7EzD8tLkpbA4jAnSHPn4ISBng3H3swmmS8265zu1PFPlslsJ0INgt8Q3z5w9AZoNUi%7EzebmvFCe1J2q

model-00001-of-00037.safetensors:   1%|          | 21.0M/3.76G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/0d/ee/0deecfb2b45a159252701d3bf1cb56f185d30cc6e44e2f5899d977f0c84fe140/60ea416808b446ee22e0912107f316e17075848bb5baa4630cf78244ae6f1b07?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00037.safetensors%3B+filename%3D%22model-00001-of-00037.safetensors%22%3B&Expires=1718680823&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODY4MDgyM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBkL2VlLzBkZWVjZmIyYjQ1YTE1OTI1MjcwMWQzYmYxY2I1NmYxODVkMzBjYzZlNDRlMmY1ODk5ZDk3N2YwYzg0ZmUxNDAvNjBlYTQxNjgwOGI0NDZlZTIyZTA5MTIxMDdmMzE2ZTE3MDc1ODQ4YmI1YmFhNDYzMGNmNzgyNDRhZTZmMWIwNz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H58SrjQGUkdy9BspopdcSPYtthtuMp6xvp6YQfOv-kqv-DMMNI2zOBH6vRR5ZqimOHL3PgTZo-7FvABDmFyYGxNGUoUm7P2Qu4T%7Ek-UJIH2YkrUjQgdD1M-vP0uU7%7EzD8tLkpbA4jAnSHPn4ISBng3H3swmmS8265zu1PFPlslsJ0INgt8Q3z5w9AZoNUi%7EzebmvFCe1J2q

model-00001-of-00037.safetensors:   2%|1         | 62.9M/3.76G [00:00<?, ?B/s]

In [3]:
!python3 app.py --server.port 8501

CUDA available: True
Traceback (most recent call last):
  File "/notebooks/app.py", line 7, in <module>
    model = model.to("cuda")
NameError: name 'model' is not defined


In [5]:
!pip install flask 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.1 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify, render_template
from transformers import AutoTokenizer, AutoModelForCausalLM

app = Flask(__name__)

# โหลดโมเดลและ tokenizer
tokenizer = AutoTokenizer.from_pretrained("KBTG-Labs/THaLLE-0.1-7B-fa")
model = AutoModelForCausalLM.from_pretrained("KBTG-Labs/THaLLE-0.1-7B-fa")

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/generate', methods=['POST'])
def generate():
    text = request.form['input_text']
    inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=200, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return jsonify({'generated_text': generated_text})

if __name__ == '__main__':
    app.run(port=8501)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8501
Press CTRL+C to quit
127.0.0.1 - - [14/Jun/2024 23:58:37] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2024 00:02:32] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2024 00:05:00] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2024 00:06:26] "POST /generate HTTP/1.1" 200 -
